In [ ]:
import csv
import pandas as pd
from pathlib import Path

In [ ]:
def clean_day_ahead_data(file_path,delim=";",onlyDK2=True,timezone = "DK",save_as_new_file = False):
    """
    Reduce unnecessary columns (anything but DK2,UTC time zone)
    reverse the data frame, since it goes backwards in comparison to Syslab
    """
    if onlyDK2 == True: 
        region = ["DK2"]
    else: region = ["DK1","DK2"]
        
    # Read the CSV file
    try:
        df = pd.read_csv(file_path,delimiter=delim,decimal=",")
    except Exception as e:
        print(f"Error reading file: {e}")
        return
    

    df_DK = df[df["PriceArea"].isin(region)]
    df_DK = df_DK.drop(columns=["PriceArea","DayAheadPriceEUR"])

    if timezone == "DK":
        df_timecorrected = df_DK.drop(columns=["TimeUTC"])
    else: df_timecorrected = df_DK.drop(columns=["TimeDK"])

    df_timecorrected = df_timecorrected.rename(columns={"TimeDK":"ts"},)
    df_reversed = df_timecorrected.iloc[::-1].reset_index(drop=True) # reverse time so it moves forward

    if save_as_new_file == True: 
        df_reversed.to_csv(f'{file_path[:-4]}_cleaned.csv', index=False, sep =",")

    return df_reversed